# Field-level NDVI Time Series
In this notebook we will use Descartes Labs `Workflows` to build an interactive map to **query the time series of vegetative health for any field in the world**.  The field boundary is drawn by the user, and the NDVI time series from Sentinel-2 is returned.  This allows the user to assess **field-level vegetative health** or determine **crop planting patterns such as winter cover crops** in an interactive, visual way.

You can run the following cells using `Shift-Enter`.

## Import packages

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# import packages
import descarteslabs.workflows as wf
from field_ndvi import FieldNDVI

## Define the _Sentinel-2_ imagery that will display on the map

In [ ]:
s2 = wf.ImageCollection.from_id(
    "sentinel-2:L1C", start_datetime="2018-03-01", end_datetime="2018-04-01"
)

s2 = s2.filter(lambda img: img.properties["cloud_fraction"] <= 0.05)
s2 = s2.pick_bands("nir red green").median(axis="images")
s2.visualize(
    "Sentinel-2 NRG", scales=[[0, 0.3], [0, 0.3], [0, 0.3]], checkerboard=False
)

## Create the `FieldNDVI` object, a custom widget defined in `field_ndvi.py`
This is the widget that will compute the NDVI time series of your field of interest.  The date range for this time series takes on a default value but can be adjusted once you make the map below.

In [ ]:
ndvi_timeseries = FieldNDVI(wf.map)

## Define the map center and zoom level.
We choose a field in Iowa that appears to have a **winter cover crop** in early 2018 but not in early 2019.

In [ ]:
wf.map.center = 42.2474, -94.41329  # Iowa
wf.map.zoom = 14

## Finally, display and interact with the map.
Select the polygon draw tool on the left and draw a polygon over your field of interest.  This will trigger a comptutation on the Descartes Labs `Workflows` backend to compute the mean NDVI in this field, which will then display embedded in the map.

In [ ]:
wf.map

### If desired, return the NDVI time series as a pandas `dataframe`.
You must first draw an AOI in the map above.  Then uncomment and run the line below.

In [ ]:
# ndvi_timeseries.df